# Лабораторна робота №2. Наука про дані: підготовчий етап.

## Мета роботи: ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції.

### Виконала студентка ФБ-23 - Присєвок Оксана

#### Хід виконання

In [1]:
import os
import pandas as pd
from datetime import datetime
import urllib3

In [2]:
def main():
    http = urllib3.PoolManager()
    for i in range(1, 28):
        url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2020&type=Mean'
        response = http.request('GET', url)
        text = response.data
        now = datetime.now()
        date_and_time_time = now.strftime("%d%m%Y%H%M%S")
        with open(f"{PATH}NOAA_ID{i}_{date_and_time_time}.csv", 'wb') as f:
            f.write(text)
            
        if i == 1:
            df = obl_read(f"{PATH}NOAA_ID{i}_{date_and_time_time}.csv", i)
        else:
            df = pd.concat([df, obl_read(f"{PATH}NOAA_ID{i}_{date_and_time_time}.csv", i)])
        # print(obl_read(f"{PATH}NOAA_ID{i}_{date_and_time_time}.csv", i).head())

    print("VHI is downloaded...")
    df.to_csv('complete_dataset.csv', encoding='utf-8')

In [3]:
names = {1: 'Вінницька', 
         2: 'Волинська',
         3: 'Дніпропетровська',
         4: 'Донецька',
         5: 'Житомирська',
         6: 'Закарпатська',
         7: 'Запорізька',
         8: 'Івано-Франківська',
         9: 'Київська',
         10: 'Кіровоградська',
         11: 'Луганська',
         12: 'Львівська',
         13: 'Миколаївська',
         14: 'Одеська',
         15: 'Полтавська',
         16: 'Рівенська',
         17: 'Сумська',
         18: 'Тернопільська',
         19: 'Харківська',
         20: 'Херсонська',
         21: 'Хмельницька',
         22: 'Черкаська',
         23: 'Чернівецька',
         24: 'Чернігівська',
         25: 'Республіка Крим'}
def change_region(df):
    df['Region'] = df['Region'].map(names)
    return df

In [4]:
def obl_read(filename, i):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(filename, header=1, names=headers)  # read csv, create df
    df.drop('empty', inplace=True, axis=1)  # drop empty column
    df.drop(df.tail(1).index, inplace=True)  # drop last row
    df['Year'] = df['Year'].replace(['<tt><pre>1982'], '1982')  # delete tags from first value
    df = df.drop(df.loc[df['VHI'] == -1].index)  # drop all nan values
    # df['region'] = i  # service reg number
    # df['region'].replace({i: names[i]}, inplace=True)  # task reg number
    return df

##### Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу): Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [5]:
def min_max_vhi(df, region_name, year):
    filtered_df = df[(df["region"] == region_name) & (df["Year"] == year)]
    
    if filtered_df.empty:
        print("No data found for the specified area and year.")
        return
    
    min_vhi = filtered_df["VHI"].min()
    max_vhi = filtered_df["VHI"].max()
    
    min_vhi_row = filtered_df[filtered_df["VHI"] == min_vhi]
    max_vhi_row = filtered_df[filtered_df["VHI"] == max_vhi]
    
    print("Minimum VHI:")
    print("Value:", min_vhi)
    print("Data:")
    print(min_vhi_row)

    print("Maximum VHI:")
    print("Value:", max_vhi)
    print("Data:")
    print(max_vhi_row)
min_max_vhi(df, Київська, 2005)


NameError: name 'df' is not defined

##### Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25); Аналогічно для помірних посух

In [ ]:
def find_drought_years(df, threshold_percent):
    # Визначення порогових значень для екстремальних і помірних посух
    extreme_drought_threshold = 10  # Порогове значення для екстремальних посух
    moderate_drought_threshold = 20  # Порогове значення для помірних посух
    
    # Список унікальних років у даних
    years = df['Year'].unique()
    drought_years = []
    
    # Проходження по всім рокам
    for year in years:
        # Вибір даних за певний рік
        year_data = df[df['Year'] == year]
        
        # Кількість областей, які відповідають умовам екстремальної та помірної посух
        extreme_drought_count = year_data[year_data['VHI'] <= extreme_drought_threshold]['region'].nunique()
        moderate_drought_count = year_data[year_data['VHI'] <= moderate_drought_threshold]['region'].nunique()
        
        # Перевірка, чи більше вказаного відсотка областей мають екстремальні або помірні посухи
        if extreme_drought_count / 25 >= threshold_percent:
            drought_years.append((year, 'Extreme Drought'))
        elif moderate_drought_count / 25 >= threshold_percent:
            drought_years.append((year, 'Moderate Drought'))
    
    return drought_years
